In [55]:

import random
import statistics
# Class to define players who play the prisoner's dilemma.  
# Each player has an open function that returns a boolean value
# and a behaviour function that returns a boolean value.
# The open function is called on the first move of the game.
# The behaviour function is called on all subsequent moves.
# The open function is called with no arguments.
# The behaviour function can be called with the following optional arguments:
#     - the player's previous move(s)
#     - the opponent's previous move(s)    
class Player:
    def __init__(self, open_func, response_func):
        self.open_func = open_func
        self.response_func = response_func
        self.player_moves = []
        self.opponent_moves = []
        self.score = 0
    
    def open(self):
        return self.open_func(self)
    
    def response(self, *args, **kwargs):
        return self.response_func(self, *args, **kwargs)
    
    def add_moves(self, player_move, opponent_move):
        self.player_moves.append(player_move)
        self.opponent_moves.append(opponent_move)
    
    def last_n_player_moves(self, n):
        return self.player_moves[-n:]  
    
    def last_n_opponent_moves(self, n):
        return self.opponent_moves[-n:]
    
    def calculate_score(self):
        self.score = 0
        # True is cooperate, False is defect
        for i in range(len(self.player_moves)):
            if self.player_moves[i] == True and self.opponent_moves[i] == True:
                self.score += 1
            elif self.player_moves[i] == True and self.opponent_moves[i] == False:
                self.score += 0
            elif self.player_moves[i] == False and self.opponent_moves[i] == True:
                self.score += 3
            elif self.player_moves[i] == False and self.opponent_moves[i] == False:
                self.score += 2
        return self.score
        
    def reset(self):
        self.player_moves = []
        self.opponent_moves = []


In [56]:
def simulate_game(player1, player2, rounds=1000):
    """
    Simulates a prisoner's dilemma game between two players over a given number of rounds.
    
    The first round calls each player's `open` method,
    while subsequent rounds call the `response` method.
    
    The function returns a tuple of the final scores: (score_player1, score_player2).
    """
    # Reset the move history before the game begins.
    player1.reset()
    player2.reset()
    
    for i in range(rounds):
        if i == 0:
            move1 = player1.open()
            move2 = player2.open()
        else:          
            move1 = player1.response()
            move2 = player2.response()
        
        # Update both players' histories.
        player1.add_moves(player_move=move1, opponent_move=move2)
        player2.add_moves(player_move=move2, opponent_move=move1)
    
    score1 = player1.calculate_score()
    score2 = player2.calculate_score()
    
    return score1, score2

def simulate_series(player1, player2, num_games=10, rounds=1000):
    """
    Simulate a series of games between player1 and player2.
    
    For half the games, player1 starts (simulate_game(player1, player2)).
    For the other half, player2 starts (simulate_game(player2, player1)).
    
    The function computes and returns the mean score and standard deviation
    for each player as a tuple:
        ((mean_score_player1, sd_player1), (mean_score_player2, sd_player2))
    """
    scores_p1 = []
    scores_p2 = []
    
    # For half the games, player1 starts.
    for _ in range(num_games // 2):
        s1, s2 = simulate_game(player1, player2, rounds)
        scores_p1.append(s1)
        scores_p2.append(s2)
    
    # For the other half, player2 starts.
    for _ in range(num_games // 2):
        s1, s2 = simulate_game(player2, player1, rounds)
        # Here s1 is the score of the player who was passed as player2 in the original order,
        # so we swap the assignment.
        scores_p1.append(s2)
        scores_p2.append(s1)
    
    mean_p1 = statistics.mean(scores_p1)
    sd_p1 = statistics.pstdev(scores_p1)
    mean_p2 = statistics.mean(scores_p2)
    sd_p2 = statistics.pstdev(scores_p2)
    
    return (mean_p1, sd_p1), (mean_p2, sd_p2)

In [94]:
always_defect = Player(
    open_func=lambda self: False,
    response_func=lambda self : False
)

mostly_nice_player = Player(
    open_func=lambda self: random.choice([True] * 50 + [False]),
    response_func=lambda self : random.choice([True] * 50 + [False])
)

random_player = Player(
    open_func=lambda self: random.choice([True, False]),
    response_func=lambda self : random.choice([True, False])
)

social_player = Player(
    open_func=lambda self: True,
    response_func=lambda self: False if self.last_n_opponent_moves(1)[0] == False else True
)

never_forgiving_player = Player(
    open_func=lambda self: True,
    response_func=lambda self: False if False in self.opponent_moves else True
)

hard_to_forgiving_player = Player(
    open_func=lambda self: True,
    response_func=lambda self: False if False in self.last_n_opponent_moves(3) else True
)

In [ ]:
players = {
    "always_defect": always_defect,
    "mostly_nice_player": mostly_nice_player,
    "random_player": random_player,
    "social_player": social_player,
    "never_forgiving_player": never_forgiving_player,
    "hard_to_forgiving_player": hard_to_forgiving_player
}

import pandas as pd

# List of player names and their corresponding objects (assume `players` dictionary is defined)
player_names = list(players.keys())

# Create an empty DataFrame with player names as both rows and columns.
results_matrix = pd.DataFrame(index=player_names, columns=player_names)

# Fill the diagonal with a placeholder
for name in player_names:
    results_matrix.loc[name, name] = 0

# Run round-robin tournament: each pair of players play a series of games.
for i, name1 in enumerate(player_names):
    for j, name2 in enumerate(player_names):
        if i < j:  # Ensure each pair is played once.
            (mean1, sd1), (mean2, sd2) = simulate_series(players[name1], players[name2], num_games=10, rounds=1000)
            # Store the result: row player gets mean1, and column player gets mean2.
            results_matrix.loc[name1, name2] = f"{mean1:.2f} ± {sd1:.2f}"
            results_matrix.loc[name2, name1] = f"{mean2:.2f} ± {sd2:.2f}"

print(results_matrix)

